In [58]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers
import tensorflow.keras.backend as K
import os
import efficientnet.tfkeras as efn
from keras.utils import np_utils
import optuna
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from tensorflow.keras import activations

In [59]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
gpus = tf.config.list_physical_devices('GPU'); print(gpus)
if len(gpus)==1: strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
else: strategy = tf.distribute.MirroredStrategy()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [60]:
tf.config.optimizer.set_experimental_options({"auto_mixed_precision": True})
print('Mixed precision enabled')

Mixed precision enabled


In [61]:
X = np.load("../input/pcb-image-dataset1/xtrain.npy")
y = np.load("../input/pcb-image-dataset1/ytrain.npy")

In [62]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [64]:
conv_base = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

In [65]:
nos = X.shape[0]
bs = int(nos/8)


numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,1920))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)


In [66]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

In [ ]:
samples.shape, labels.shape

In [68]:
def optimize(trial):
    n_epochs = trial.suggest_int('n_epochs',20,100)
    n_learningrate = trial.suggest_uniform("n_learningrate",0.0001,0.1)
    n_bs = trial.suggest_int('n_bs',8,64)
    n_patience = trial.suggest_int('n_patience',1,20)
    
    
    accuracies = []
    f = 1

    for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        K.clear_session()
        model = models.Sequential()
        model.add(layers.Conv2D(64, (2, 2), input_shape=(9, 9, 1920)))
        model.add(layers.BatchNormalization())
        model.add(layers.Activation(activations.relu))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(256, (2, 2)))
        model.add(layers.BatchNormalization())
        model.add(layers.Activation(activations.relu))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(64, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))

        model.compile(optimizer = optimizers.Adam(lr = n_learningrate),loss = 'binary_crossentropy',metrics = ['accuracy'])
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=n_patience, restore_best_weights=True)]

        print('Fold {}'.format(f))
#         with strategy.scope():
        history = model.fit(X_train, y_train, epochs = n_epochs, batch_size = n_bs, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)

        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1
        
    zero_acc = 0
    one_acc = 0

    for i in accuracies:
        zero_acc+= i[0]
        one_acc+=i[1]

    zero_acc/=8
    one_acc/=8

    return zero_acc+one_acc


In [69]:
from functools import partial
optimization_function = partial(optimize)

In [70]:
study = optuna.create_study(direction='maximize')

In [71]:
study.optimize(optimization_function, n_trials = 15 )

Fold 1
Epoch 1/58


[W 2020-07-20 17:39:33,613] Setting status of trial#0 as TrialState.FAIL because of the following error: ValueError("in user code:\n\n    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *\n        outputs = self.distribute_strategy.run(\n    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/one_device_strategy.py:182 run  **\n        return super(OneDeviceStrategy, self).run(fn, args, kwargs, options)\n    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run\n        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)\n    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica\n        return self._call_for_each_replica(fn, args, kwargs)\n    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/one_device_strategy.py:362 _call_for_each_replica\n        return fn(*args, **kwargs)\n 

ValueError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/one_device_strategy.py:182 run  **
        return super(OneDeviceStrategy, self).run(fn, args, kwargs, options)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/distribute/one_device_strategy.py:362 _call_for_each_replica
        return fn(*args, **kwargs)
    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:541 train_step  **
        self.

In [ ]:
study.best_params

In [ ]:
Xtest = np.load("../input/pcb-image-dataset1/xtest.npy")
ytest = np.load("../input/pcb-image-dataset1/ytest.npy")

In [ ]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagentest.fit(Xtest)

In [ ]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    Xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(testnos,9,9,1920))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

In [ ]:
test_samples.shape, test_labels.shape

In [ ]:
study.best_params

In [ ]:
samples.shape, labels.shape

In [ ]:
K.clear_session()
accuracies = []
model = models.Sequential()
model.add(layers.Conv2D(64, (2, 2), input_shape=(9, 9, 1920)))
model.add(layers.BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (2, 2)))
model.add(layers.BatchNormalization())
model.add(layers.Activation(activations.relu))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer = optimizers.Adam(lr = study.best_params['n_learningrate']),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=study.best_params['n_patience'], restore_best_weights=True)]

# with strategy.scope():
historytest = model.fit(samples, labels, epochs = study.best_params['n_epochs'], batch_size = study.best_params['n_bs'], validation_data = (test_samples, test_labels), shuffle = True, callbacks = my_callbacks)

test_preds = model.predict(test_samples, batch_size = 20)
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

accuracies.append([confusion_matrix(test_labels,test_predclass)[0][0]/(test_labels == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(test_labels == 1.0).sum()])

In [ ]:
accuracies

In [ ]:
historytest.history['loss'], historytest.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(historytest.history['loss'],'r')
plt.plot(historytest.history['val_loss'],'g')
plt.show()